# Analisis Penjualan Retail

Proyek ini menganalisis data penjualan dari sebuah toko retail.

In [1]:
import sqlite3
import pandas as pd

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('retail_sales.db')
cursor = conn.cursor()

## Membuat Tabel

In [2]:
# Membuat tabel Produk
cursor.execute('''
CREATE TABLE Produk (
    produk_id INTEGER PRIMARY KEY,
    nama TEXT NOT NULL,
    kategori TEXT NOT NULL,
    harga REAL NOT NULL
)
''')

# Membuat tabel Pelanggan
cursor.execute('''
CREATE TABLE Pelanggan (
    pelanggan_id INTEGER PRIMARY KEY,
    nama TEXT NOT NULL,
    email TEXT NOT NULL
)
''')

# Membuat tabel Penjualan
cursor.execute('''
CREATE TABLE Penjualan (
    penjualan_id INTEGER PRIMARY KEY,
    produk_id INTEGER,
    pelanggan_id INTEGER,
    tanggal TEXT NOT NULL,
    jumlah INTEGER NOT NULL,
    total_harga REAL NOT NULL,
    FOREIGN KEY (produk_id) REFERENCES Produk(produk_id),
    FOREIGN KEY (pelanggan_id) REFERENCES Pelanggan(pelanggan_id)
)
''')

conn.commit()

## Mengisi Data

In [3]:
# Mengisi data ke tabel Produk
produk_data = [
    (1, 'Produk A', 'Kategori 1', 10000),
    (2, 'Produk B', 'Kategori 1', 15000),
    (3, 'Produk C', 'Kategori 2', 20000)
]
cursor.executemany('INSERT INTO Produk VALUES (?,?,?,?)', produk_data)

# Mengisi data ke tabel Pelanggan
pelanggan_data = [
    (1, 'Andi', 'andi@example.com'),
    (2, 'Budi', 'budi@example.com'),
    (3, 'Cici', 'cici@example.com')
]
cursor.executemany('INSERT INTO Pelanggan VALUES (?,?,?)', pelanggan_data)

# Mengisi data ke tabel Penjualan
penjualan_data = [
    (1, 1, 1, '2023-01-01', 2, 20000),
    (2, 2, 1, '2023-01-05', 1, 15000),
    (3, 3, 2, '2023-01-10', 1, 20000),
    (4, 1, 3, '2023-02-01', 3, 30000)
]
cursor.executemany('INSERT INTO Penjualan VALUES (?,?,?,?,?,?)', penjualan_data)

conn.commit()

## Analisis Data

In [4]:
# Produk terlaris
query = '''
SELECT p.nama, SUM(jumlah) as total_terjual 
FROM Penjualan 
JOIN Produk p ON Penjualan.produk_id = p.produk_id 
GROUP BY p.nama 
ORDER BY total_terjual DESC
LIMIT 1
'''
pd.read_sql_query(query, conn)

In [5]:
# Pelanggan paling setia
query = '''
SELECT pl.nama, COUNT(penjualan_id) as total_transaksi 
FROM Penjualan 
JOIN Pelanggan pl ON Penjualan.pelanggan_id = pl.pelanggan_id 
GROUP BY pl.nama 
ORDER BY total_transaksi DESC
LIMIT 1
'''
pd.read_sql_query(query, conn)

In [6]:
# Total penjualan per bulan
query = '''
SELECT strftime('%Y-%m', tanggal) as bulan, SUM(total_harga) as total_penjualan 
FROM Penjualan 
GROUP BY bulan 
ORDER BY bulan
'''
pd.read_sql_query(query, conn)

## Penutupan Koneksi

In [7]:
# Menutup koneksi ke database
conn.close()